In [1]:
#Make sure dependencies are installed
import cobra
import BOFdat as bd
import pandas as pd
from Bio import SeqIO
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from cobra.util.solver import linear_reaction_coefficients

In [2]:
sb.set_style('whitegrid')

In [3]:
#Give the path to each file as function parameters
#Genome file in BioPython supported format (.faa, .fna) and GenBank file 
#also in BioPython supported format (.gb, .gbff)
genome = 'Ecoli_DNA.fna'
genbank = 'Ecoli_K12_MG1655.gbff'

#OMICs data as a 2 column csv file, gene and abundance
transcriptomic = 'transcriptomic.csv'
proteomic = 'proteomic.csv'

#Lipidomic abundances and conversion to model identifier
lipidomic_abundances = 'lipidomic_abundances.csv'
lipidomic_conversion = 'lipidomic_conversion.csv'

#Metabolomic abundances and conversion to model identifier
metabolomic_abundances = 'metabolomic_abundances.csv'
metabolomic_conversion = 'metabolomic_conversion.csv'

#Growth data on different carbon sources, uptake and secretion rates
maintenance = 'maintenance.csv'

#The model for which the coefficients are generated
model = 'iML1515.json'

In [4]:
ecoli = cobra.io.load_json_model(model)

In [5]:
#calculate and plot fold change
old_biomass = {}
for m in ecoli.reactions.Ec_biomass_iML1515_core_75p37M.reactants:
    old_biomass[m.id] = abs(ecoli.reactions.Ec_biomass_iML1515_core_75p37M.get_coefficient(m))

mean_error = []
mega_df = pd.DataFrame(index=np.linspace(0.01,0.9,num=50),columns=['dna','rna','protein','lipid'])
for ratio in np.linspace(0.01,0.9,num=50):
    dna_coeff = bd.dna.generate_coefficients(genome,model,DNA_RATIO=ratio)
    rna_coeff = bd.rna.generate_coefficients(genbank,model,transcriptomic,TOTAL_RNA_RATIO=ratio)
    prot_coeff = bd.protein.generate_coefficients(genbank,model,proteomic,PROTEIN_RATIO=ratio)
    lipid_coeff = bd.lipid.generate_coefficients(lipidomic_abundances,lipidomic_conversion,model,LIPID_RATIO=ratio)
    #metab_coeff = bd.metabolite.generate_coefficients_from_experimental_data(metabolomic_abundances,metabolomic_conversion,model,METAB_RATIO=ratio)
    new_biomass = [dna_coeff, rna_coeff, prot_coeff, lipid_coeff]
    #Generate a new biomass objective function by adding all the components together
    mean_error = []
    for i in range(len(new_biomass)):
        d = new_biomass[i]
        df1 = pd.DataFrame({'new_coefficient':d.values(),'metab':[m.id for m in d.keys()]})
        df2 = pd.DataFrame({'original_coefficient':old_biomass.values(),'metab': old_biomass.keys()})
        df3 = pd.merge(left=df1,right=df2,how='outer',on='metab')
        #Remove data that is absurdely divergent
        cut_off = (df3.new_coefficient.median() + df3.original_coefficient.median())/2 + (df3.new_coefficient.std()+df3.original_coefficient.std())
        
        #Get all data below cut-off
        df3['difference'] = abs(df3.new_coefficient[df3['new_coefficient']<cut_off]) - abs(df3.original_coefficient[df3['original_coefficient']<cut_off])
        df3['percent_error'] = abs(df3['difference']/df3.original_coefficient[df3['original_coefficient']<cut_off])*100
        data = df3[df3['percent_error'].isnull() == False].percent_error
        mean_error.append(data.mean())
    
    data_df.loc[ratio] = mean_error
    
data_df

NameError: name 'data_df' is not defined

In [ ]:
#Plot
ax = mega_df.plot()
#Plot the minimum value for each of them
plt.plot()
# zoom-in / limit the view to different portions of the data
ax.set_ylim(0., 100.)  # outliers only
plt.ylabel('Percent error')
plt.xlabel('Weight fraction')
plt.show()

In [ ]:
min_error, weight_fraction_min = [],[]
for col in mega_df.columns:
    min_error.append(mega_df[col].min())
    weight_fraction_min.append(mega_df[col].idxmin())
result_df = pd.DataFrame({'min_error':min_error,'weight_fraction_min':weight_fraction_min},index=['dna','rna','protein','lipid'])
result_df.plot(kind='bar')
plt.show()

In [80]:
result_df


,min_error,weight_fraction_min
dna,7.081532,0.054949
rna,14.431379,0.306667
protein,24.710127,0.630303
lipid,22.363994,0.216768


In [72]:
mega_df.protein.idxmin()

0.63030303030303036

In [59]:
mega_df.to_csv('results_fig.csv')

,dna,rna,protein,lipid
0.010000,80.5127,96.7831,98.5779,96.3309
0.018990,62.9939,93.8911,97.2994,93.0323
0.027980,45.475,90.9991,96.0209,89.7338
0.036970,27.9562,88.1071,94.7424,86.4353
0.045960,10.4373,85.2151,93.4639,83.1368
0.054949,7.08153,82.3231,92.1854,79.8383
0.063939,24.6004,79.4311,90.9069,76.5397
0.072929,42.1192,76.5391,89.6284,73.2412
0.081919,59.6381,73.6471,88.3499,69.9427
0.090909,77.1569,70.7551,87.0714,66.6442


In [37]:
old_biomass

{<Metabolite dgtp_c at 0x7fd25beffc90>: 0.027017,
 <Metabolite fad_c at 0x7fd25bf24410>: 0.000223,
 <Metabolite succoa_c at 0x7fd25bf35350>: 9.8e-05,
 <Metabolite his__L_c at 0x7fd25bf36290>: 0.094738,
 <Metabolite ribflv_c at 0x7fd25bf362d0>: 0.000223,
 <Metabolite so4_c at 0x7fd25bf36350>: 0.004338,
 <Metabolite pe161_p at 0x7fd25bf36dd0>: 0.075214,
 <Metabolite 2fe2s_c at 0x7fd25bf36f50>: 2.6e-05,
 <Metabolite gtp_c at 0x7fd25bf48150>: 0.215096,
 <Metabolite btn_c at 0x7fd25bf48290>: 2e-06,
 <Metabolite val__L_c at 0x7fd25bf488d0>: 0.423162,
 <Metabolite pro__L_c at 0x7fd25bf5b250>: 0.221055,
 <Metabolite coa_c at 0x7fd25bf6d050>: 0.000576,
 <Metabolite asn__L_c at 0x7fd25bf6d090>: 0.241055,
 <Metabolite leu__L_c at 0x7fd25bf6d150>: 0.450531,
 <Metabolite 4fe4s_c at 0x7fd25bf6dc90>: 0.00026,
 <Metabolite cl_c at 0x7fd25bf7f390>: 0.005205,
 <Metabolite murein5px4p_p at 0x7fd25bf92190>: 0.013894,
 <Metabolite pheme_c at 0x7fd25bf92cd0>: 0.000223,
 <Metabolite 2ohph_c at 0x7fd25bf92d90

In [25]:
df3.new_coefficient.median()

0.0025491558328838634

In [20]:
for i,row in df3.iterrows():
    print(row.metab, row.new_coefficient, row.original_coefficient)

(u'5mta_c', 0.00041621734216333415, nan)
(u'ump_p', 0.0075570898269380635, nan)
(u'alltn_c', 0.00018634418894550607, nan)
(u'acglu_c', 0.0005151385466297172, nan)
(u'ru5p__D_c', 0.00046698581827805854, nan)
(u'ptrc_e', 0.07602716531618797, nan)
(u'cit_c', 0.0008966263382877431, nan)
(u'udpglcur_c', 0.0004600208040941871, nan)
(u'acon_C_c', 0.00034600570946104946, nan)
(u'spmd_p', 0.0007415111234063207, nan)
(u'ahcys_c', 7.376475975594234e-05, nan)
(u'alaala_c', 0.002572334755730239, nan)
(u'cbasp_c', 0.006913039729684365, nan)
(u'ade_p', 0.00015285613203150461, nan)
(u'uacgam_c', 0.0019174488689623086, nan)
(u'quin_e', 0.0279834419726804, nan)
(u'pser__L_e', 0.00018246741863738456, nan)
(u'cdpdhdecg_c', 0.0001228001776139871, nan)
(u'nadph_c', 0.00047612751073745414, nan)
(u'gdpfuc_c', 0.0004032004990218537, nan)
(u'g6p_c', 0.0009381372805471971, nan)
(u'acgam1p_e', 2.6132246900691972e-05, nan)
(u'adphep_DD_c', 0.001029871665236765, nan)
(u'sl26da_c', 0.03814413584952316, nan)
(u'pep_c

In [ ]:
'''
labels = df3[df3['fold_change'].isnull() == False].metab
x_pos = np.arange(len(labels))
plt.bar(x_pos,data.sort_values())
plt.show()

plt.clf()
plt.plot(np.linspace(0.1,0.9,num=10),mean_error,'bo')
plt.xlabel('Protein ratio')
plt.ylabel('Mean fold change')
plt.show()
'''